<a href="https://colab.research.google.com/github/shawn-cavalieri/panic_at_the_desktop_final_project/blob/master/stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("hello")

hello


In [2]:
pip install quandl

In [0]:
# this program predicts stock prices by using machine learning models

# install dependencies

import quandl
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [5]:
# get stock data

df = quandl.get("WIKI/FB")

# take a look at the data

df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2012-05-18,42.05,45.00,38.00,38.2318,573576400.0,0.0,1.0,42.05,45.00,38.00,38.2318,573576400.0
2012-05-21,36.53,36.66,33.00,34.0300,168192700.0,0.0,1.0,36.53,36.66,33.00,34.0300,168192700.0
2012-05-22,32.61,33.59,30.94,31.0000,101786600.0,0.0,1.0,32.61,33.59,30.94,31.0000,101786600.0
2012-05-23,31.37,32.50,31.36,32.0000,73600000.0,0.0,1.0,31.37,32.50,31.36,32.0000,73600000.0
2012-05-24,32.95,33.21,31.77,33.0300,50237200.0,0.0,1.0,32.95,33.21,31.77,33.0300,50237200.0


In [6]:
# get adjusted close price

df = df[['Adj. Close']]

df.head()

,Adj. Close
Date,
2012-05-18,38.2318
2012-05-21,34.0300
2012-05-22,31.0000
2012-05-23,32.0000
2012-05-24,33.0300


In [27]:
# variable for predicting 'n' days into the future

forecast_out = 30

# create another column (the target or dependent variable) shifted 'n' units up

df['Prediction'] = df[['Adj. Close']].shift(-forecast_out)

df.tail()

,Adj. Close,Prediction
Date,,
2018-03-21,169.39,NaN
2018-03-22,164.89,NaN
2018-03-23,159.39,NaN
2018-03-26,160.06,NaN
2018-03-27,152.19,NaN


In [12]:
## create the independent data set (x)

# convert dataframe to a numpy array

X = np.array(df.drop(['Prediction'],1))

# remove the last 'n' rows

X = X[:-forecast_out]

print(X)

[[ 38.2318]
 [ 34.03  ]
 [ 31.    ]
 ...
 [164.89  ]
 [159.39  ]
 [160.06  ]]


In [13]:
### create the dependent data set (y) ###

# convert the dataframe to a numpy array (all of the values including NANs)

y = np.array(df['Prediction'])

# get all of the y values except the last 'n' rows

y = y[:-forecast_out]

print(y)

[ 34.03  31.    32.   ... 159.39 160.06 152.19]


In [0]:
# split the data into 80% training and 20% testing

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [15]:
# create and train the Support Vector Machine (regressor)

svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)

svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [17]:
# testing model: Score returns the coefficient of determination R^2 of the prediction
# the best possible score is 1.0

svm_confidence = svr_rbf.score(x_test, y_test)

print("svm confidence: ", svm_confidence)

svm confidence:  0.9987374245361974


In [18]:
# create and train the Linear Regression Model

lr = LinearRegression()

# train the model

lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [19]:
# testing model: Score returns the coefficient of determination R^2 of the prediction
# the best possible score is 1.0

lr_confidence = lr.score(x_test, y_test)

print("lr confidence: ", lr_confidence)

lr confidence:  0.9988038004584465


In [28]:
# set x_forecast equal to the last 30 rows of the original data set from Adj. Close column

x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]

print(x_forecast)

[[173.15]
 [179.52]
 [179.96]
 [177.36]
 [176.01]
 [177.91]
 [178.99]
 [183.29]
 [184.93]
 [181.46]
 [178.32]
 [175.94]
 [176.62]
 [180.4 ]
 [179.78]
 [183.71]
 [182.34]
 [185.23]
 [184.76]
 [181.88]
 [184.19]
 [183.86]
 [185.09]
 [172.56]
 [168.15]
 [169.39]
 [164.89]
 [159.39]
 [160.06]
 [152.19]]


In [36]:
# print the linear regression model predictions for the next 'n' days

lr_prediction = lr.predict(x_forecast)



# predict support vector regressor model predictions for the next 'n' days

svm_prediction = svr_rbf.predict(x_forecast)

print(lr_prediction)
print(svm_prediction)

[173.14875646 179.51466144 179.95437858 177.35605002 176.00691788
 177.90569644 178.98500216 183.28223786 184.92118357 181.45341429
 178.31543287 175.93696288 176.61652573 180.39409572 179.7744943
 183.70196786 182.33284858 185.22099071 184.75129286 181.87314429
 184.18165929 183.85187143 185.08108071 172.55913574 168.15197076
 169.39117361 164.89406648 159.39760221 160.06717149 152.2022308 ]
[172.39889531 179.02870131 179.26306813 177.27202712 176.49378106
 177.87232857 178.80173841 182.46344899 184.49999122 181.43399973
 178.30823875 176.46776631 176.72502563 179.67979393 179.15151028
 182.49214769 182.48932371 185.30630552 184.06246336 182.07943027
 182.91808226 182.57176332 184.92889683 171.7346952  169.17684922
 169.8327601  165.15209575 159.48049597 159.91779812 152.58304307]
